In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/patchtest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 27s 267ms/step - loss: 5.2871 - binary_crossentropy: 0.4622 - jaccard_coef_int: 0.1272 - binary_accuracy: 0.8332 - val_loss: 7.2008 - val_binary_crossentropy: 0.7820 - val_jaccard_coef_int: 8.9992e-16 - val_binary_accuracy: 0.8011
Epoch 2/50
100/100 [==============================] - 13s 132ms/step - loss: 4.2508 - binary_crossentropy: 0.5959 - jaccard_coef_int: 0.2311 - binary_accuracy: 0.8400 - val_loss: 8.1027 - val_binary_crossentropy: 0.9710 - val_jaccard_coef_int: 8.9992e-16 - val_binary_accuracy: 0.8011
Epoch 3/50
100/100 [==============================] - 14s 143ms/step - loss: 3.9775 - binary_crossentropy: 0.6433 - jaccard_coef_int: 0.2453 - binary_accuracy: 0.8485 - val_loss: 8.6535 - val_binary_crossentropy: 1.0862 - val_jaccard_coef_int: 8.8910e-16 - val_binary_accuracy: 0.7986
Epoch 4/50
100/100 [==============================] - 13s 133ms/step - loss: 3.8858 - binary_crossentropy: 0.7233 - jaccard_coef_int: 0.2590 - bi

100/100 [==============================] - 13s 134ms/step - loss: 2.2622 - binary_crossentropy: 0.8893 - jaccard_coef_int: 0.4845 - binary_accuracy: 0.9045 - val_loss: 3.5102 - val_binary_crossentropy: 1.4459 - val_jaccard_coef_int: 0.3638 - val_binary_accuracy: 0.8439
Epoch 31/50
100/100 [==============================] - 13s 130ms/step - loss: 2.1979 - binary_crossentropy: 0.8633 - jaccard_coef_int: 0.4936 - binary_accuracy: 0.9075 - val_loss: 3.0147 - val_binary_crossentropy: 1.2947 - val_jaccard_coef_int: 0.4611 - val_binary_accuracy: 0.8603
Epoch 32/50
100/100 [==============================] - 13s 133ms/step - loss: 2.1765 - binary_crossentropy: 0.8667 - jaccard_coef_int: 0.4939 - binary_accuracy: 0.9086 - val_loss: 3.1959 - val_binary_crossentropy: 1.2963 - val_jaccard_coef_int: 0.4235 - val_binary_accuracy: 0.8543
Epoch 33/50
100/100 [==============================] - 13s 130ms/step - loss: 2.2108 - binary_crossentropy: 0.8914 - jaccard_coef_int: 0.4954 - binary_accuracy: 0.906

Epoch 7/50
100/100 [==============================] - 13s 132ms/step - loss: 2.4940 - binary_crossentropy: 0.0667 - jaccard_coef_int: 0.1165 - binary_accuracy: 0.9896 - val_loss: 2.7900 - val_binary_crossentropy: 0.1190 - val_jaccard_coef_int: 0.0810 - val_binary_accuracy: 0.9837
Epoch 8/50
100/100 [==============================] - 14s 140ms/step - loss: 2.4969 - binary_crossentropy: 0.0773 - jaccard_coef_int: 0.1126 - binary_accuracy: 0.9892 - val_loss: 2.6897 - val_binary_crossentropy: 0.1125 - val_jaccard_coef_int: 0.0918 - val_binary_accuracy: 0.9834
Epoch 9/50
100/100 [==============================] - 13s 130ms/step - loss: 2.4890 - binary_crossentropy: 0.0833 - jaccard_coef_int: 0.1130 - binary_accuracy: 0.9892 - val_loss: 3.1632 - val_binary_crossentropy: 0.0993 - val_jaccard_coef_int: 0.0520 - val_binary_accuracy: 0.9878
Epoch 10/50
100/100 [==============================] - 13s 132ms/step - loss: 2.3988 - binary_crossentropy: 0.0823 - jaccard_coef_int: 0.1209 - binary_accura

...

Epoch 9/50
100/100 [==============================] - 17s 171ms/step - loss: 3.7148 - binary_crossentropy: 0.5424 - jaccard_coef_int: 0.0999 - binary_accuracy: 0.9143 - val_loss: 3.9840 - val_binary_crossentropy: 0.5415 - val_jaccard_coef_int: 0.0701 - val_binary_accuracy: 0.9177
Epoch 10/50
100/100 [==============================] - 16s 165ms/step - loss: 3.6161 - binary_crossentropy: 0.5368 - jaccard_coef_int: 0.1056 - binary_accuracy: 0.9167 - val_loss: 4.2109 - val_binary_crossentropy: 0.5498 - val_jaccard_coef_int: 0.0585 - val_binary_accuracy: 0.9147
Epoch 11/50
100/100 [==============================] - 16s 164ms/step - loss: 3.6064 - binary_crossentropy: 0.5645 - jaccard_coef_int: 0.1077 - binary_accuracy: 0.9160 - val_loss: 8.6322 - val_binary_crossentropy: 0.4544 - val_jaccard_coef_int: 3.0897e-15 - val_binary_accuracy: 0.9413
Epoch 12/50
100/100 [==============================] - 17s 168ms/step - loss: 3.5038 - binary_crossentropy: 0.5668 - jaccard_coef_int: 0.1157 - binary_

100/100 [==============================] - 17s 167ms/step - loss: 2.1764 - binary_crossentropy: 0.6088 - jaccard_coef_int: 0.2846 - binary_accuracy: 0.9430 - val_loss: 3.8685 - val_binary_crossentropy: 0.9252 - val_jaccard_coef_int: 0.0776 - val_binary_accuracy: 0.9182
Epoch 39/50
100/100 [==============================] - 16s 164ms/step - loss: 2.1296 - binary_crossentropy: 0.6022 - jaccard_coef_int: 0.2934 - binary_accuracy: 0.9442 - val_loss: 4.2354 - val_binary_crossentropy: 1.1232 - val_jaccard_coef_int: 0.0667 - val_binary_accuracy: 0.9046
Epoch 40/50
100/100 [==============================] - 17s 165ms/step - loss: 2.1239 - binary_crossentropy: 0.6052 - jaccard_coef_int: 0.2937 - binary_accuracy: 0.9444 - val_loss: 4.5581 - val_binary_crossentropy: 0.7439 - val_jaccard_coef_int: 0.0311 - val_binary_accuracy: 0.9319
Epoch 41/50
100/100 [==============================] - 17s 166ms/step - loss: 2.1256 - binary_crossentropy: 0.6139 - jaccard_coef_int: 0.2952 - binary_accuracy: 0.943

Epoch 15/50
100/100 [==============================] - 17s 165ms/step - loss: 3.1414 - binary_crossentropy: 0.5825 - jaccard_coef_int: 0.1417 - binary_accuracy: 0.9270 - val_loss: 3.7654 - val_binary_crossentropy: 0.6954 - val_jaccard_coef_int: 0.0924 - val_binary_accuracy: 0.9137
Epoch 16/50
100/100 [==============================] - 16s 164ms/step - loss: 3.0780 - binary_crossentropy: 0.5837 - jaccard_coef_int: 0.1462 - binary_accuracy: 0.9288 - val_loss: 3.8720 - val_binary_crossentropy: 0.7294 - val_jaccard_coef_int: 0.0886 - val_binary_accuracy: 0.9098
Epoch 17/50
100/100 [==============================] - 16s 164ms/step - loss: 2.9980 - binary_crossentropy: 0.5755 - jaccard_coef_int: 0.1513 - binary_accuracy: 0.9317 - val_loss: 4.3306 - val_binary_crossentropy: 0.7393 - val_jaccard_coef_int: 0.0496 - val_binary_accuracy: 0.9172
Epoch 18/50
100/100 [==============================] - 17s 166ms/step - loss: 2.9712 - binary_crossentropy: 0.5860 - jaccard_coef_int: 0.1550 - binary_acc

Epoch 21/50
100/100 [==============================] - 16s 164ms/step - loss: 2.3950 - binary_crossentropy: 0.0898 - jaccard_coef_int: 0.1540 - binary_accuracy: 0.9879 - val_loss: 4.0838 - val_binary_crossentropy: 0.0161 - val_jaccard_coef_int: 0.5614 - val_binary_accuracy: 0.9977
Epoch 22/50
100/100 [==============================] - 17s 167ms/step - loss: 2.3220 - binary_crossentropy: 0.0926 - jaccard_coef_int: 0.1750 - binary_accuracy: 0.9879 - val_loss: 5.4397 - val_binary_crossentropy: 0.0136 - val_jaccard_coef_int: 0.5667 - val_binary_accuracy: 0.9980
Epoch 23/50
100/100 [==============================] - 16s 164ms/step - loss: 2.4939 - binary_crossentropy: 0.0913 - jaccard_coef_int: 0.1729 - binary_accuracy: 0.9881 - val_loss: 4.9579 - val_binary_crossentropy: 0.0487 - val_jaccard_coef_int: 0.0484 - val_binary_accuracy: 0.9931
Epoch 24/50
100/100 [==============================] - 17s 167ms/step - loss: 2.6761 - binary_crossentropy: 0.0969 - jaccard_coef_int: 0.1471 - binary_acc

100/100 [==============================] - 17s 174ms/step - loss: 2.5740 - binary_crossentropy: 0.1067 - jaccard_coef_int: 0.1511 - binary_accuracy: 0.9865 - val_loss: 5.4068 - val_binary_crossentropy: 0.0129 - val_jaccard_coef_int: 0.5673 - val_binary_accuracy: 0.9984
0.9879127528901734
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99   1098608
        1.0       0.25      0.28      0.26      8592

avg / total       0.99      0.99      0.99   1107200





In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/patchtest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 69s 693ms/step - loss: 5.9630 - binary_crossentropy: 0.6756 - jaccard_coef_int: 0.2401 - binary_accuracy: 0.7512 - val_loss: 9.2975 - val_binary_crossentropy: 2.0917 - val_jaccard_coef_int: 0.2545 - val_binary_accuracy: 0.5000
Epoch 2/50
100/100 [==============================] - 14s 138ms/step - loss: 4.6894 - binary_crossentropy: 0.7921 - jaccard_coef_int: 0.3678 - binary_accuracy: 0.7817 - val_loss: 4.6881 - val_binary_crossentropy: 1.1920 - val_jaccard_coef_int: 0.2110 - val_binary_accuracy: 0.8057
Epoch 3/50
100/100 [==============================] - 14s 139ms/step - loss: 4.5288 - binary_crossentropy: 0.9293 - jaccard_coef_int: 0.3754 - binary_accuracy: 0.7843 - val_loss: 5.3639 - val_binary_crossentropy: 1.2411 - val_jaccard_coef_int: 0.3297 - val_binary_accuracy: 0.7355
Epoch 4/50
100/100 [==============================] - 14s 138ms/step - loss: 4.3583 - binary_crossentropy: 0.9984 - jaccard_coef_int: 0.3850 - binary_accurac

Epoch 7/50
100/100 [==============================] - 15s 147ms/step - loss: 4.2023 - binary_crossentropy: 0.7286 - jaccard_coef_int: 0.1242 - binary_accuracy: 0.8730 - val_loss: 11.5205 - val_binary_crossentropy: 8.5679 - val_jaccard_coef_int: 0.1348 - val_binary_accuracy: 0.4103
Epoch 8/50
100/100 [==============================] - 14s 139ms/step - loss: 4.0324 - binary_crossentropy: 0.7409 - jaccard_coef_int: 0.1303 - binary_accuracy: 0.8800 - val_loss: 5.4795 - val_binary_crossentropy: 0.8967 - val_jaccard_coef_int: 0.0392 - val_binary_accuracy: 0.8595
Epoch 9/50
100/100 [==============================] - 15s 153ms/step - loss: 3.9166 - binary_crossentropy: 0.7598 - jaccard_coef_int: 0.1374 - binary_accuracy: 0.8831 - val_loss: 8.6713 - val_binary_crossentropy: 0.7751 - val_jaccard_coef_int: 1.6242e-15 - val_binary_accuracy: 0.8883
Epoch 10/50
100/100 [==============================] - 15s 150ms/step - loss: 3.9590 - binary_crossentropy: 0.7949 - jaccard_coef_int: 0.1317 - binary_a

Epoch 36/50
100/100 [==============================] - 14s 140ms/step - loss: 3.4660 - binary_crossentropy: 1.0646 - jaccard_coef_int: 0.1755 - binary_accuracy: 0.8940 - val_loss: 5.5167 - val_binary_crossentropy: 1.2202 - val_jaccard_coef_int: 0.0282 - val_binary_accuracy: 0.8777
Epoch 37/50
100/100 [==============================] - 14s 141ms/step - loss: 3.4473 - binary_crossentropy: 1.0689 - jaccard_coef_int: 0.1759 - binary_accuracy: 0.8949 - val_loss: 11.9745 - val_binary_crossentropy: 1.1486 - val_jaccard_coef_int: 1.6250e-15 - val_binary_accuracy: 0.8883
Epoch 38/50
100/100 [==============================] - 14s 141ms/step - loss: 3.4413 - binary_crossentropy: 1.0891 - jaccard_coef_int: 0.1811 - binary_accuracy: 0.8940 - val_loss: 5.5402 - val_binary_crossentropy: 1.2327 - val_jaccard_coef_int: 0.0254 - val_binary_accuracy: 0.8832
Epoch 39/50
100/100 [==============================] - 14s 139ms/step - loss: 3.4446 - binary_crossentropy: 1.0952 - jaccard_coef_int: 0.1808 - binar

...

Epoch 2/50
100/100 [==============================] - 18s 179ms/step - loss: 2.5578 - binary_crossentropy: 0.1454 - jaccard_coef_int: 0.1923 - binary_accuracy: 0.9636 - val_loss: 3.3224 - val_binary_crossentropy: 0.0846 - val_jaccard_coef_int: 0.0552 - val_binary_accuracy: 0.9822
Epoch 3/50
100/100 [==============================] - 17s 174ms/step - loss: 2.3298 - binary_crossentropy: 0.1818 - jaccard_coef_int: 0.2038 - binary_accuracy: 0.9640 - val_loss: 2.5263 - val_binary_crossentropy: 0.2059 - val_jaccard_coef_int: 0.1456 - val_binary_accuracy: 0.9696
Epoch 4/50
100/100 [==============================] - 18s 184ms/step - loss: 2.2907 - binary_crossentropy: 0.2077 - jaccard_coef_int: 0.2007 - binary_accuracy: 0.9637 - val_loss: 2.6068 - val_binary_crossentropy: 0.2482 - val_jaccard_coef_int: 0.1405 - val_binary_accuracy: 0.9634
Epoch 5/50
100/100 [==============================] - 17s 173ms/step - loss: 2.1731 - binary_crossentropy: 0.2181 - jaccard_coef_int: 0.2184 - binary_accurac

Epoch 8/50
100/100 [==============================] - 18s 175ms/step - loss: 2.8942 - binary_crossentropy: 0.9396 - jaccard_coef_int: 0.6134 - binary_accuracy: 0.8506 - val_loss: 2.9643 - val_binary_crossentropy: 0.9963 - val_jaccard_coef_int: 0.6011 - val_binary_accuracy: 0.8471
Epoch 9/50
100/100 [==============================] - 18s 176ms/step - loss: 2.8790 - binary_crossentropy: 0.9627 - jaccard_coef_int: 0.6139 - binary_accuracy: 0.8514 - val_loss: 3.2327 - val_binary_crossentropy: 1.1262 - val_jaccard_coef_int: 0.5548 - val_binary_accuracy: 0.8349
Epoch 10/50
100/100 [==============================] - 17s 174ms/step - loss: 2.8477 - binary_crossentropy: 0.9784 - jaccard_coef_int: 0.6210 - binary_accuracy: 0.8526 - val_loss: 2.7646 - val_binary_crossentropy: 0.9801 - val_jaccard_coef_int: 0.6367 - val_binary_accuracy: 0.8560
Epoch 11/50
100/100 [==============================] - 17s 173ms/step - loss: 2.8358 - binary_crossentropy: 0.9759 - jaccard_coef_int: 0.6272 - binary_accur

Epoch 14/50
100/100 [==============================] - 14s 139ms/step - loss: 0.6463 - binary_crossentropy: 0.0432 - jaccard_coef_int: 0.6480 - binary_accuracy: 0.9932 - val_loss: 1.0892 - val_binary_crossentropy: 0.0871 - val_jaccard_coef_int: 0.4283 - val_binary_accuracy: 0.9861
Epoch 15/50
100/100 [==============================] - 14s 139ms/step - loss: 0.7320 - binary_crossentropy: 0.0456 - jaccard_coef_int: 0.6422 - binary_accuracy: 0.9928 - val_loss: 1.0807 - val_binary_crossentropy: 0.1409 - val_jaccard_coef_int: 0.4267 - val_binary_accuracy: 0.9860
Epoch 16/50
100/100 [==============================] - 14s 143ms/step - loss: 0.5865 - binary_crossentropy: 0.0457 - jaccard_coef_int: 0.6661 - binary_accuracy: 0.9935 - val_loss: 1.0799 - val_binary_crossentropy: 0.1061 - val_jaccard_coef_int: 0.4289 - val_binary_accuracy: 0.9860
Epoch 17/50
100/100 [==============================] - 15s 149ms/step - loss: 0.6097 - binary_crossentropy: 0.0514 - jaccard_coef_int: 0.6533 - binary_acc

Epoch 20/50
100/100 [==============================] - 17s 168ms/step - loss: 4.1390 - binary_crossentropy: 0.1783 - jaccard_coef_int: 0.0427 - binary_accuracy: 0.9709 - val_loss: 4.5403 - val_binary_crossentropy: 0.0912 - val_jaccard_coef_int: 0.0048 - val_binary_accuracy: 0.9851
Epoch 21/50
100/100 [==============================] - 18s 184ms/step - loss: 3.8425 - binary_crossentropy: 0.1244 - jaccard_coef_int: 0.0438 - binary_accuracy: 0.9792 - val_loss: 4.5837 - val_binary_crossentropy: 0.5705 - val_jaccard_coef_int: 0.0410 - val_binary_accuracy: 0.9154
Epoch 22/50
100/100 [==============================] - 17s 168ms/step - loss: 3.4393 - binary_crossentropy: 0.1176 - jaccard_coef_int: 0.0560 - binary_accuracy: 0.9822 - val_loss: 4.6283 - val_binary_crossentropy: 0.1942 - val_jaccard_coef_int: 0.0144 - val_binary_accuracy: 0.9669
Epoch 23/50
100/100 [==============================] - 17s 168ms/step - loss: 3.4765 - binary_crossentropy: 0.1329 - jaccard_coef_int: 0.0547 - binary_acc

100/100 [==============================] - 17s 169ms/step - loss: 3.5981 - binary_crossentropy: 0.3246 - jaccard_coef_int: 0.0640 - binary_accuracy: 0.9691 - val_loss: 3.1320 - val_binary_crossentropy: 0.1471 - val_jaccard_coef_int: 0.0598 - val_binary_accuracy: 0.9814
Epoch 50/50
100/100 [==============================] - 17s 173ms/step - loss: 4.7865 - binary_crossentropy: 0.1718 - jaccard_coef_int: 0.0337 - binary_accuracy: 0.9710 - val_loss: 5.4585 - val_binary_crossentropy: 0.1120 - val_jaccard_coef_int: 0.0014 - val_binary_accuracy: 0.9863
0.9923022037572254
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00   1100808
        1.0       0.01      0.00      0.00      6392

avg / total       0.99      0.99      0.99   1107200





In [ ]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/patchtest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 107s 1s/step - loss: 1.9121 - binary_crossentropy: 0.0781 - jaccard_coef_int: 0.6324 - binary_accuracy: 0.9666 - val_loss: 0.6675 - val_binary_crossentropy: 0.0462 - val_jaccard_coef_int: 0.6342 - val_binary_accuracy: 0.9905
Epoch 2/50
100/100 [==============================] - 18s 179ms/step - loss: 0.6203 - binary_crossentropy: 0.0225 - jaccard_coef_int: 0.7546 - binary_accuracy: 0.9952 - val_loss: 0.6350 - val_binary_crossentropy: 0.0504 - val_jaccard_coef_int: 0.6599 - val_binary_accuracy: 0.9910
Epoch 3/50
100/100 [==============================] - 19s 188ms/step - loss: 0.5250 - binary_crossentropy: 0.0220 - jaccard_coef_int: 0.7440 - binary_accuracy: 0.9953 - val_loss: 0.5999 - val_binary_crossentropy: 0.0549 - val_jaccard_coef_int: 0.6554 - val_binary_accuracy: 0.9909
Epoch 4/50
100/100 [==============================] - 18s 179ms/step - loss: 0.4771 - binary_crossentropy: 0.0265 - jaccard_coef_int: 0.7390 - binary_accuracy:

Epoch 7/50
100/100 [==============================] - 17s 173ms/step - loss: 4.5549 - binary_crossentropy: 1.2107 - jaccard_coef_int: 0.3015 - binary_accuracy: 0.7921 - val_loss: 4.5568 - val_binary_crossentropy: 1.1980 - val_jaccard_coef_int: 0.2502 - val_binary_accuracy: 0.8030
Epoch 8/50
100/100 [==============================] - 18s 178ms/step - loss: 4.5404 - binary_crossentropy: 1.2554 - jaccard_coef_int: 0.3062 - binary_accuracy: 0.7918 - val_loss: 4.6066 - val_binary_crossentropy: 1.3653 - val_jaccard_coef_int: 0.2671 - val_binary_accuracy: 0.7956
Epoch 9/50
100/100 [==============================] - 17s 173ms/step - loss: 4.4735 - binary_crossentropy: 1.2768 - jaccard_coef_int: 0.3083 - binary_accuracy: 0.7953 - val_loss: 4.6958 - val_binary_crossentropy: 1.3434 - val_jaccard_coef_int: 0.3030 - val_binary_accuracy: 0.7817
Epoch 10/50
100/100 [==============================] - 17s 173ms/step - loss: 4.4137 - binary_crossentropy: 1.3030 - jaccard_coef_int: 0.3136 - binary_accura

Epoch 13/50
100/100 [==============================] - 14s 143ms/step - loss: 3.3941 - binary_crossentropy: 0.9872 - jaccard_coef_int: 0.3071 - binary_accuracy: 0.8623 - val_loss: 4.0403 - val_binary_crossentropy: 1.1716 - val_jaccard_coef_int: 0.2260 - val_binary_accuracy: 0.8408
Epoch 14/50
100/100 [==============================] - 14s 144ms/step - loss: 3.3861 - binary_crossentropy: 1.0167 - jaccard_coef_int: 0.3143 - binary_accuracy: 0.8613 - val_loss: 3.6517 - val_binary_crossentropy: 1.1100 - val_jaccard_coef_int: 0.2951 - val_binary_accuracy: 0.8485
Epoch 15/50
100/100 [==============================] - 14s 143ms/step - loss: 3.3655 - binary_crossentropy: 1.0384 - jaccard_coef_int: 0.3163 - binary_accuracy: 0.8621 - val_loss: 3.7573 - val_binary_crossentropy: 1.1924 - val_jaccard_coef_int: 0.2788 - val_binary_accuracy: 0.8454
Epoch 16/50
100/100 [==============================] - 14s 143ms/step - loss: 3.3211 - binary_crossentropy: 1.0469 - jaccard_coef_int: 0.3230 - binary_acc

Epoch 19/50
100/100 [==============================] - 17s 171ms/step - loss: 1.8535 - binary_crossentropy: 0.5926 - jaccard_coef_int: 0.5335 - binary_accuracy: 0.9240 - val_loss: 2.3005 - val_binary_crossentropy: 0.7665 - val_jaccard_coef_int: 0.4788 - val_binary_accuracy: 0.9026
Epoch 20/50
100/100 [==============================] - 17s 172ms/step - loss: 1.9265 - binary_crossentropy: 0.6218 - jaccard_coef_int: 0.5192 - binary_accuracy: 0.9214 - val_loss: 6.9467 - val_binary_crossentropy: 2.5249 - val_jaccard_coef_int: 0.2543 - val_binary_accuracy: 0.6541
Epoch 21/50
100/100 [==============================] - 17s 170ms/step - loss: 1.8583 - binary_crossentropy: 0.5995 - jaccard_coef_int: 0.5233 - binary_accuracy: 0.9249 - val_loss: 2.2608 - val_binary_crossentropy: 0.7799 - val_jaccard_coef_int: 0.4906 - val_binary_accuracy: 0.9038
Epoch 22/50
100/100 [==============================] - 18s 176ms/step - loss: 1.8198 - binary_crossentropy: 0.6108 - jaccard_coef_int: 0.5381 - binary_acc

Epoch 25/50
100/100 [==============================] - 14s 144ms/step - loss: 1.1992 - binary_crossentropy: 0.1809 - jaccard_coef_int: 0.4254 - binary_accuracy: 0.9793 - val_loss: 1.4907 - val_binary_crossentropy: 0.2194 - val_jaccard_coef_int: 0.3387 - val_binary_accuracy: 0.9747
Epoch 26/50
100/100 [==============================] - 14s 145ms/step - loss: 1.1558 - binary_crossentropy: 0.1845 - jaccard_coef_int: 0.4423 - binary_accuracy: 0.9796 - val_loss: 1.5050 - val_binary_crossentropy: 0.2242 - val_jaccard_coef_int: 0.3362 - val_binary_accuracy: 0.9745
Epoch 27/50
100/100 [==============================] - 14s 144ms/step - loss: 1.1230 - binary_crossentropy: 0.1825 - jaccard_coef_int: 0.4522 - binary_accuracy: 0.9800 - val_loss: 1.4945 - val_binary_crossentropy: 0.2256 - val_jaccard_coef_int: 0.3323 - val_binary_accuracy: 0.9756
Epoch 28/50
100/100 [==============================] - 15s 151ms/step - loss: 1.0794 - binary_crossentropy: 0.1778 - jaccard_coef_int: 0.4652 - binary_acc

Epoch 2/50
100/100 [==============================] - 19s 195ms/step - loss: 3.6074 - binary_crossentropy: 0.0845 - jaccard_coef_int: 0.1059 - binary_accuracy: 0.9791 - val_loss: 9.5757 - val_binary_crossentropy: 2.5511 - val_jaccard_coef_int: 0.0167 - val_binary_accuracy: 0.6624
Epoch 3/50
100/100 [==============================] - 19s 195ms/step - loss: 3.3032 - binary_crossentropy: 0.1016 - jaccard_coef_int: 0.1093 - binary_accuracy: 0.9791 - val_loss: 6.4602 - val_binary_crossentropy: 0.0462 - val_jaccard_coef_int: 5.3223e-14 - val_binary_accuracy: 0.9927
Epoch 4/50
 16/100 [===>..........................] - ETA: 15s - loss: 2.8156 - binary_crossentropy: 0.0927 - jaccard_coef_int: 0.1356 - binary_accuracy: 0.9817

In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/patchtest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[..., verbose=1, steps_per_epoch=100)`


Epoch 1/50
100/100 [==============================] - 28s 279ms/step - loss: 3.8207 - binary_crossentropy: 0.2497 - jaccard_coef_int: 0.2745 - binary_accuracy: 0.9102 - val_loss: 5.7922 - val_binary_crossentropy: 0.3705 - val_jaccard_coef_int: 8.6155e-04 - val_binary_accuracy: 0.9114
Epoch 2/50
100/100 [==============================] - 19s 194ms/step - loss: 2.3358 - binary_crossentropy: 0.2409 - jaccard_coef_int: 0.4596 - binary_accuracy: 0.9256 - val_loss: 6.2463 - val_binary_crossentropy: 0.4450 - val_jaccard_coef_int: 0.0022 - val_binary_accuracy: 0.9100
Epoch 3/50
100/100 [==============================] - 20s 197ms/step - loss: 2.0752 - binary_crossentropy: 0.2557 - jaccard_coef_int: 0.4768 - binary_accuracy: 0.9288 - val_loss: 2.4586 - val_binary_crossentropy: 0.3244 - val_jaccard_coef_int: 0.3202 - val_binary_accuracy: 0.9227
Epoch 4/50
100/100 [==============================] - 19s 193ms/step - loss: 2.0845 - binary_crossentropy: 0.3042 - jaccard_coef_int: 0.4715 - binary_acc

100/100 [==============================] - 19s 189ms/step - loss: 0.6543 - binary_crossentropy: 0.1347 - jaccard_coef_int: 0.7754 - binary_accuracy: 0.9760 - val_loss: 0.9575 - val_binary_crossentropy: 0.2642 - val_jaccard_coef_int: 0.6937 - val_binary_accuracy: 0.9638
Epoch 31/50
100/100 [==============================] - 19s 190ms/step - loss: 0.6152 - binary_crossentropy: 0.1250 - jaccard_coef_int: 0.7855 - binary_accuracy: 0.9776 - val_loss: 0.5164 - val_binary_crossentropy: 0.1009 - val_jaccard_coef_int: 0.8127 - val_binary_accuracy: 0.9817
Epoch 32/50
100/100 [==============================] - 19s 190ms/step - loss: 0.6148 - binary_crossentropy: 0.1261 - jaccard_coef_int: 0.7873 - binary_accuracy: 0.9775 - val_loss: 0.5191 - val_binary_crossentropy: 0.1069 - val_jaccard_coef_int: 0.8152 - val_binary_accuracy: 0.9814
Epoch 33/50
100/100 [==============================] - 19s 187ms/step - loss: 0.5836 - binary_crossentropy: 0.1192 - jaccard_coef_int: 0.7962 - binary_accuracy: 0.978

Epoch 7/50
100/100 [==============================] - 16s 164ms/step - loss: 1.3417 - binary_crossentropy: 0.1298 - jaccard_coef_int: 0.5045 - binary_accuracy: 0.9649 - val_loss: 1.3869 - val_binary_crossentropy: 0.1401 - val_jaccard_coef_int: 0.5148 - val_binary_accuracy: 0.9606
Epoch 8/50
100/100 [==============================] - 16s 164ms/step - loss: 1.2499 - binary_crossentropy: 0.1300 - jaccard_coef_int: 0.5268 - binary_accuracy: 0.9668 - val_loss: 1.0792 - val_binary_crossentropy: 0.1099 - val_jaccard_coef_int: 0.6057 - val_binary_accuracy: 0.9691
Epoch 9/50
100/100 [==============================] - 16s 164ms/step - loss: 1.1351 - binary_crossentropy: 0.1224 - jaccard_coef_int: 0.5569 - binary_accuracy: 0.9698 - val_loss: 1.2466 - val_binary_crossentropy: 0.1527 - val_jaccard_coef_int: 0.5397 - val_binary_accuracy: 0.9639
Epoch 10/50
100/100 [==============================] - 16s 164ms/step - loss: 1.0706 - binary_crossentropy: 0.1197 - jaccard_coef_int: 0.5731 - binary_accura

...

100/100 [==============================] - 16s 162ms/step - loss: 0.5720 - binary_crossentropy: 0.1178 - jaccard_coef_int: 0.7875 - binary_accuracy: 0.9799 - val_loss: 0.4521 - val_binary_crossentropy: 0.0749 - val_jaccard_coef_int: 0.8261 - val_binary_accuracy: 0.9846
Epoch 41/50
100/100 [==============================] - 16s 163ms/step - loss: 0.5378 - binary_crossentropy: 0.1083 - jaccard_coef_int: 0.7984 - binary_accuracy: 0.9812 - val_loss: 0.4088 - val_binary_crossentropy: 0.0636 - val_jaccard_coef_int: 0.8411 - val_binary_accuracy: 0.9863
Epoch 42/50
100/100 [==============================] - 16s 162ms/step - loss: 0.5269 - binary_crossentropy: 0.1068 - jaccard_coef_int: 0.8044 - binary_accuracy: 0.9814 - val_loss: 0.4225 - val_binary_crossentropy: 0.0679 - val_jaccard_coef_int: 0.8328 - val_binary_accuracy: 0.9859
Epoch 43/50
100/100 [==============================] - 16s 163ms/step - loss: 0.4979 - binary_crossentropy: 0.0993 - jaccard_coef_int: 0.8135 - binary_accuracy: 0.982

100/100 [==============================] - 19s 195ms/step - loss: 0.7818 - binary_crossentropy: 0.1125 - jaccard_coef_int: 0.6382 - binary_accuracy: 0.9805 - val_loss: 0.7048 - val_binary_crossentropy: 0.0684 - val_jaccard_coef_int: 0.6252 - val_binary_accuracy: 0.9869
Epoch 17/50
100/100 [==============================] - 20s 201ms/step - loss: 0.7483 - binary_crossentropy: 0.1092 - jaccard_coef_int: 0.6525 - binary_accuracy: 0.9812 - val_loss: 1.0284 - val_binary_crossentropy: 0.1452 - val_jaccard_coef_int: 0.5200 - val_binary_accuracy: 0.9778
Epoch 18/50
100/100 [==============================] - 19s 194ms/step - loss: 0.7332 - binary_crossentropy: 0.1074 - jaccard_coef_int: 0.6585 - binary_accuracy: 0.9815 - val_loss: 0.7907 - val_binary_crossentropy: 0.0847 - val_jaccard_coef_int: 0.5817 - val_binary_accuracy: 0.9856
Epoch 19/50
100/100 [==============================] - 19s 195ms/step - loss: 0.7043 - binary_crossentropy: 0.1019 - jaccard_coef_int: 0.6676 - binary_accuracy: 0.982

100/100 [==============================] - 19s 191ms/step - loss: 0.3480 - binary_crossentropy: 0.0472 - jaccard_coef_int: 0.8204 - binary_accuracy: 0.9913 - val_loss: 0.4048 - val_binary_crossentropy: 0.0439 - val_jaccard_coef_int: 0.7657 - val_binary_accuracy: 0.9922
Epoch 46/50
100/100 [==============================] - 20s 199ms/step - loss: 0.3391 - binary_crossentropy: 0.0464 - jaccard_coef_int: 0.8258 - binary_accuracy: 0.9914 - val_loss: 0.3205 - val_binary_crossentropy: 0.0300 - val_jaccard_coef_int: 0.8069 - val_binary_accuracy: 0.9941
Epoch 47/50
100/100 [==============================] - 19s 191ms/step - loss: 0.3537 - binary_crossentropy: 0.0508 - jaccard_coef_int: 0.8196 - binary_accuracy: 0.9909 - val_loss: 0.8579 - val_binary_crossentropy: 0.1669 - val_jaccard_coef_int: 0.5834 - val_binary_accuracy: 0.9805
Epoch 48/50
100/100 [==============================] - 19s 191ms/step - loss: 0.3522 - binary_crossentropy: 0.0510 - jaccard_coef_int: 0.8202 - binary_accuracy: 0.990

Epoch 22/50
100/100 [==============================] - 17s 167ms/step - loss: 0.8613 - binary_crossentropy: 0.1601 - jaccard_coef_int: 0.7480 - binary_accuracy: 0.9663 - val_loss: 1.5576 - val_binary_crossentropy: 0.3171 - val_jaccard_coef_int: 0.5630 - val_binary_accuracy: 0.9398
Epoch 23/50
100/100 [==============================] - 17s 166ms/step - loss: 0.8265 - binary_crossentropy: 0.1548 - jaccard_coef_int: 0.7552 - binary_accuracy: 0.9678 - val_loss: 0.7257 - val_binary_crossentropy: 0.1202 - val_jaccard_coef_int: 0.7966 - val_binary_accuracy: 0.9712
Epoch 24/50
100/100 [==============================] - 17s 172ms/step - loss: 0.8565 - binary_crossentropy: 0.1659 - jaccard_coef_int: 0.7472 - binary_accuracy: 0.9664 - val_loss: 0.6751 - val_binary_crossentropy: 0.1133 - val_jaccard_coef_int: 0.8081 - val_binary_accuracy: 0.9735
Epoch 25/50
100/100 [==============================] - 17s 166ms/step - loss: 0.7913 - binary_crossentropy: 0.1514 - jaccard_coef_int: 0.7654 - binary_acc

             precision    recall  f1-score   support

        0.0       0.99      0.98      0.99    980995
        1.0       0.89      0.95      0.92    126205

avg / total       0.98      0.98      0.98   1107200



